# Moneza-Loan-Approval-Automation


## Data Cleaning and Preprocessing

### 1. Import Libraries and Functions

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import plotly.io as pio
pio.renderers.default = 'notebook'

# Import custom functions from the src folder
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from visualization import create_bar_chart
from utils import get_categorical_columns,get_numerical_columns,convert_to_data_type,get_boolean_columns

from ydata_profiling import ProfileReport


### 2. Read Data

In [ ]:
# Load the Excel file
file_path = "../data/raw/TWINO_task_data_scientist.xlsx"

data = pd.read_excel(file_path,sheet_name="Data")
data.profile_report()

### 3. Data Cleaning

#### 3.1 Standardize Data Types and Formats

In [ ]:
categorical_columns = get_categorical_columns(data)
boolean_columns=get_boolean_columns(data)

# Convert all text data to lowercase
data[categorical_columns] = data[categorical_columns].apply(lambda x: x.str.lower().str.strip())
data[categorical_columns] = convert_to_data_type(data,categorical_columns,"category")
data[boolean_columns] = convert_to_data_type(data,boolean_columns,"int")
data[boolean_columns] = convert_to_data_type(data,boolean_columns,"category")

#Convert AR  to category
data["AR"] = convert_to_data_type(data,"AR","category")



#### 3.2 Check for missing data

In [ ]:
# Calculate the percentage of missing values for each column in the DataFrame
null_percentage_columns = data.isna().mean() * 100
null_percentage_columns = null_percentage_columns.sort_values(ascending=False)

null_percentage_columns

##### Missing Data Findings
- **Findings:** There are no missing values in any of the columns in the dataset.
- **Action:** No action required as the dataset is complete.

#### 3.3 Check for duplicates

In [ ]:
# Check for any duplicate rows
duplicates = data.duplicated()
duplicate_count = duplicates.sum()
print(f"Number of duplicate rows: {duplicate_count}")

if duplicate_count > 0:
    duplicate_rows = data[duplicates]
    display(duplicate_rows)

##### Duplicate Data Findings
- **Findings:** There are no duplicate rows in the dataset.
- **Action:** No action required as there are no duplicates.

#### 3.3 Check for outliers

In [ ]:
#Drop application_id column
data = data.drop("application_id", axis = 1)
numerical_columns = get_numerical_columns(data)

# Interquartile Range (IQR) method
Q1 = data['warning_count'].quantile(0.25)
Q3 = data['warning_count'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
iqr_outliers = data[(data['warning_count'] < lower_bound) | (data['warning_count'] > upper_bound)]
print("Number of IQR outliers:", len(iqr_outliers))

# Z-Score method
z_scores = stats.zscore(data['warning_count'])
z_score_outliers = data[abs(z_scores) > 3]
print("Number of Z-score outliers:", len(z_score_outliers))

# Combine outliers
combined_outliers_index = iqr_outliers.index.union(z_score_outliers.index)
num_combined_outliers = len(combined_outliers_index)
print("Number of combined outliers:", num_combined_outliers)
combined_outliers = data.loc[combined_outliers_index]



In [ ]:
combined_outliers_index = iqr_outliers.index.union(z_score_outliers.index)

# Number of combined outliers
num_combined_outliers = len(combined_outliers_index)
print("Number of combined outliers:", num_combined_outliers)

# Extract the combined outliers
combined_outliers = data.loc[combined_outliers_index]
combined_outliers.warning_count.value_counts()

In [ ]:
sns.boxplot(data['warning_count'])
plt.title('Warning Count')
plt.ylabel('Values')

# Annotate outliers with reduced arrow size
for outlier in combined_outliers.itertuples():
    plt.annotate(outlier.warning_count,
                 xy=(0, outlier.warning_count),
                 xytext=(0.05, outlier.warning_count + 0.2),
                 textcoords='data',
                 arrowprops=dict(facecolor='red', arrowstyle='-|>', lw=0.5),  # Adjust arrowstyle and lw (line width)
                 fontsize=8, color='blue')

plt.savefig("../reports/figures/warning_count_outliers.png")
plt.show()

In [ ]:
create_bar_chart(combined_outliers,"AR")
plt.show()

In [ ]:
combined_outliers.warning_count.unique()

##### Outliers Findings
- **Methodology:**
  - **Interquartile Range (IQR):** Outliers are defined as data points outside 1.5 times the IQR above Q3 or below Q1. This method identified 24 outliers in the `warning_count` column.
  - **Z-Score:** Outliers are defined as data points with a Z-score greater than 3 or less than -3. This method identified 52 outliers in the `warning_count` column.
  - **Combined Approach:** Combining the outliers identified by both IQR and Z-score methods, a total of 52 unique outliers were observed.
- **Findings:**
  - 52 outliers were identified out of 9,898 records.
  - I am going by the assumption that the outlier values (14, 13, 15, 16) are  reasonable and possible in real-life scenarios. Therefore, they are treated as edge cases rather than anomalies.
- **Action:** The outliers will be retained in the dataset for further analysis.

In [ ]:
# Define a colormap
from matplotlib.colors import LinearSegmentedColormap


royalblue = LinearSegmentedColormap.from_list('royalblue', [(0, (1,1,1)), (1, (0.25,0.41,0.88))])
royalblue_r = royalblue.reversed()

In [ ]:
# Calculation of the Spearman correlation
import numpy as np
from sklearn.calibration import LabelEncoder

spearman=data.copy()
label_encoder = LabelEncoder()

# encode the categorical columns
for category in get_categorical_columns(spearman):
    if category !="payment_method":
        spearman[category]= label_encoder.fit_transform(spearman[category])
spearman=pd.get_dummies(spearman,dtype='int64')

target = 'AR'
df_ordered = pd.concat([spearman.drop(target,axis=1), spearman[target]],axis=1)
corr = df_ordered.corr(method='spearman')

# Create a mask so that we see the correlation values only once
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask,1)] = True

# Plot the heatmap correlation
plt.figure(figsize=(30,30))
sns.heatmap(corr, mask=mask, annot=True, cmap=royalblue, fmt='.2f', linewidths=0.2)
plt.show()

### 4. Save cleaned data

In [ ]:
# save as parquet to persist data types
data.to_parquet("../data/processed/data_cleaned.pqt",index=False)